# PyironFlow demo

PyironFlow provides a visual programming environment to implement and execute workflows. It uses _pyiron_workflows_ as workflow language. The graphical interface is based on the js library _xyflow_ in combination with the ipywidgets library. A workflow consists of function nodes defined as pure Python functions. Input and output of these functions can be connected to input and output channels of other nodes via edges. Examples of nodes can be found in the _pyiron_nodes_ library.

For developers only: Uncomment the following line if you have edited the .js files. Make sure that the installation is in the same directory as the js/ directory! Note that that anywidget works only with react18, not with the latest react version 19.

In [1]:
# if necessary clean cache etc.
# !npm cache clean --force
# !rm -rf node_modules package-lock.json

In [2]:
# !cd .. && npm install react@18 react-dom@18 @xyflow/react @anywidget/react elkjs


In [3]:
#!npm install @dagrejs/dagre
#!npm install file-saver

In [1]:
!npx esbuild ../js/widget.jsx --minify --format=esm --bundle --outdir=../static

⠙⠹⠸⠼Need to install the following packages:
esbuild@0.25.2
Ok to proceed? (y) 
⠙npm error canceled
⠙npm error A complete log of this run can be found in: /Users/joerg/.npm/_logs/2025-04-10T19_25_41_110Z-debug-0.log
⠙

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
%config IPCompleter.evaluation='unsafe'

In [4]:
import sys
# sys.path.remove('/Users/joerg/git_libs/pyiron_nodes')
sys.path.insert(0, '/Users/joerg/git_libs/pyiron_core')
sys.path

['/Users/joerg/git_libs/pyiron_core',
 '/Users/joerg/git_libs/pyiron_nodes',
 '/Users/joerg/git_libs/pyiron_workflow',
 '/Users/joerg/git_libs/ironflow',
 '/Users/joerg/git_libs/pyiron_core/notebooks',
 '/Users/joerg/miniforge3/envs/py12/lib/python312.zip',
 '/Users/joerg/miniforge3/envs/py12/lib/python3.12',
 '/Users/joerg/miniforge3/envs/py12/lib/python3.12/lib-dynload',
 '',
 '/Users/joerg/miniforge3/envs/py12/lib/python3.12/site-packages']

In [5]:
# import pyiron_nodes
# import pyiron_nodes.atomistic

In [6]:
import logging
logging.getLogger().setLevel(logging.WARNING)

## Generic example

The following example shows how to define nodes (functions) and how to connect them to construct a workflow. More details on how to construct nodes and workflows can be found in the documentation of _pyiron_workflows_.

In [7]:
# import sys
# from pathlib import Path
# sys.path.remove('/Users/joerg/python_projects/git_libs/pyiron_workflow')
# sys.path.insert(0, '/Users/joerg/python_projects/git_libs/pyiron-xyflow')
# sys.path

In [8]:
from pyironflow.pyironflow import PyironFlow

from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node

In [9]:
# from pyiron_nodes.atomistic.property.elastic import OutputElasticAnalysis

In [10]:
import numpy as np

@as_function_node
def linspace(x_min: int = 0, x_max: float = 2 * np.pi, n: int = 50):

    vec = np.linspace(x_min, x_max, n)
    return vec

@as_function_node(labels=['vec'])
def sin(x):
    import numpy as np
    
    vec = np.sin(x)
    return vec

@as_function_node('plot')
def plot(x, y):
    import matplotlib.pylab as plt
    
    plt.plot(x, y)
    return plt.show()

@as_macro_node(labels='structure')
def bulk_macro(element: str, cell_size: int = 1, vacancy_index: int = 0):
    from pyiron_nodes.atomistic.structure.transform import (
        CreateVacancy,
        Repeat,
    )
    from pyiron_nodes.atomistic.structure.build import Bulk

    wf = Workflow('bulk_macro')
    wf.bulk = Bulk(name=element, cubic=True)
    wf.cell = Repeat(structure=wf.bulk, repeat_scalar=cell_size)

    wf.structure = CreateVacancy(structure=wf.cell, index=vacancy_index)
    return wf.structure

In [11]:
macro = bulk_macro() #element='Al')
macro.inputs


,label,type,default,ready,value,node
0,element,str,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
1,cell_size,int,1,True,1,<pyiron_workflow.simple_workflow.Node object a...
2,vacancy_index,int,0,True,0,<pyiron_workflow.simple_workflow.Node object a...


In [12]:
macro._wf_macro.bulk.inputs

,label,type,default,ready,value,node
0,name,str,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
1,crystalstructure,str,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
2,a,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
3,c,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
4,c_over_a,int,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
5,u,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
6,orthorhombic,bool,False,True,False,<pyiron_workflow.simple_workflow.Node object a...
7,cubic,bool,False,True,True,<pyiron_workflow.simple_workflow.Node object a...


In [13]:
node_dict = macro.to_dict()
node_dict

{'label': 'bulk_macro',
 'function': '__main__.bulk_macro',
 'inputs': {'element': 'NotData'}}

In [14]:
node = Node.from_dict(node_dict)
node._wf_macro.bulk.inputs

,label,type,default,ready,value,node
0,name,str,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
1,crystalstructure,str,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
2,a,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
3,c,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
4,c_over_a,int,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
5,u,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
6,orthorhombic,bool,False,True,False,<pyiron_workflow.simple_workflow.Node object a...
7,cubic,bool,False,True,True,<pyiron_workflow.simple_workflow.Node object a...


In [15]:
macro = bulk_macro('Al', cell_size=3, vacancy_index=0)
# macro.run() # .plot3d()
macro._wf_macro.bulk.inputs

,label,type,default,ready,value,node
0,name,str,NotData,True,Al,<pyiron_workflow.simple_workflow.Node object a...
1,crystalstructure,str,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
2,a,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
3,c,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
4,c_over_a,int,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
5,u,float,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
6,orthorhombic,bool,False,True,False,<pyiron_workflow.simple_workflow.Node object a...
7,cubic,bool,False,True,True,<pyiron_workflow.simple_workflow.Node object a...


In [16]:
from pyiron_nodes.atomistic.structure.view import Plot3d

wf = Workflow('simple_workflow')
# wf.linspace = linspace(n=100)
# wf.sin = sin(x=wf.linspace)
# wf.plot = plot(x=wf.linspace, y=wf.sin)

wf.bulk_macro = bulk_macro(element='Al', cell_size=1, vacancy_index=0)
wf.plot3d = Plot3d(structure=wf.bulk_macro)

# wf.run()

In [17]:
wf.plot3d.inputs

,label,type,default,ready,value,node
0,structure,NonPrimitive,NotData,True,<pyiron_workflow.simple_workflow.Node object a...,<pyiron_workflow.simple_workflow.Node object a...
1,camera,str,orthographic,True,orthographic,<pyiron_workflow.simple_workflow.Node object a...
2,particle_size,float,1.0,True,1.0,<pyiron_workflow.simple_workflow.Node object a...
3,select_atoms,NonPrimitive,None,True,None,<pyiron_workflow.simple_workflow.Node object a...
4,view_plane,NonPrimitive,"[0, 0, 1]",True,"[0, 0, 1]",<pyiron_workflow.simple_workflow.Node object a...
5,distance_from_camera,float,1.0,True,1.0,<pyiron_workflow.simple_workflow.Node object a...


In [18]:
from pyironflow.wf_extensions import get_nodes, get_edges 
import json
from pyiron_workflow import Node

v = get_nodes(wf)# [1]['data']['target_values'][4]
# print(v)
# json.dumps(v)




In [19]:
# get_nodes(wf)[1]

The workflow _wf_ can be transferred to the visual programming interface _PyironFlow_ as shown in the cell below. To run the workflow, click on the last node (plot) and then click on the _run_ button. If you want to debug the code, you can click on any node and click run to see the output of that node. If you want to view the source code of the node, click the _source_ button.

In [20]:
wf.label = 'workflow'
pf = PyironFlow([wf])
pf.gui

In [24]:
xx

NameError: name 'xx' is not defined

In [ ]:
wf_new = pf.get_workflow()
wf_new.label = 'workflow'
pf = PyironFlow([wf_new])
pf.gui
# wf_new.run()


In [ ]:
wf_new.bulk_macro._wf_macro.bulk.inputs  # _get_non_default_input()

In [ ]:
wf_new.bulk_macro.pull()

In [ ]:
xx

In [ ]:
from pyiron_nodes.atomistic.structure.view import Plot3d
from pyiron_nodes.atomistic.structure.build import Bulk

wf = Workflow('simple_workflow')
wf.bulk = Bulk(name='Al', cell_size=1, vacancy_index=0)
wf.plot3d = Plot3d(structure=wf.bulk)

wf.run()

In [ ]:
wf_new = pf.get_workflow()

pf = PyironFlow([wf_new])
pf.gui

In [ ]:
xx

In [ ]:
wf_new = pf.get_workflow()
id(wf_new.Repeat.inputs.structure.value)

In [ ]:
wf_new = pf.get_workflow()

In [ ]:
[id(n) for n in wf_new._nodes.values()]

In [ ]:
wf_new._nodes

In [ ]:
id(wf_new.Plot3d.inputs.structure.value)

In [ ]:
wf_new = pf.get_workflow()
wf_new.Bulk.inputs

In [ ]:
id(wf_new.Repeat)

Problem: Nodes need to be ordered! At least when executed. It may be advantageous to have ordering even in ._nodes

In [ ]:
wf_new.run()

In [ ]:
wf = Workflow('structure_workflow')
element = 'Al'
cell_size = 1
vacancy_index = None

wf.bulk = Bulk(name=element, cubic=True)
wf.cell = Repeat(structure=wf.bulk, repeat_scalar=cell_size)

wf.structure = CreateVacancy(structure=wf.cell, index=vacancy_index)

wf.run()

In [ ]:
wf.label = 'workflow'
pf = PyironFlow([wf_new])
pf.gui

## Employ the node library to build more complex workflows

The _pyiron_nodes_ library collects nodes/functions from different domains. If you are using git clone to add the pyiron_nodes library, modify the following commands to add the pyiron_nodes path to your PYIRONPATH environment variable.

### Elastic constants

In [ ]:
import pyiron_nodes
import pyiron_nodes.atomistic
pyiron_nodes.atomistic


In [ ]:
from pyiron_workflow import Workflow   

import pyiron_nodes as pn

wf = Workflow('compute_elastic_constants')
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.Bulk('Pb', cubic=True)
wf.input_elastic = pn.atomistic.property.elastic.InputElasticTensor()
wf.elastic = pn.atomistic.property.elastic.ElasticConstants(structure=wf.bulk, engine=wf.engine, parameters=wf.input_elastic)
wf.output = pn.atomistic.property.elastic.OutputElasticAnalysis(dc=wf.elastic)

# wf.run()

In [ ]:
from dataclasses import asdict

wf.elastic.pull()
dc = wf.elastic.outputs.elastic.value
asdict(dc)
# type(dc)
# out = pn.atomistic.property.elastic.OutputElasticAnalysis(dc=wf.elastic.outputs.elastic.value.dataclass)

# out.pull()
# out.outputs.A2.value


In [ ]:
#wf.run()

In [ ]:
wf.draw();

In [ ]:
wf.bulk.position = (10, 0)
'position' in dir(wf.bulk)

In [ ]:
from pyironflow.pyironflow import PyironFlow

pf = PyironFlow([wf]) #, hash_nodes=True)
pf.gui

In [ ]:
from pyironflow.pyironflow import PyironFlow

pf = PyironFlow([wf]) #, hash_nodes=True)
pf.gui

### Some extra features

#### Get the nodes from the gui

In [ ]:
import json
json.loads(pf.wf_widgets[0].gui.nodes);

#### Show the workflow widget from tab_0

In [ ]:
pf.wf_widgets[0].gui

#### Get the current workflow from the gui and visualize it (test completeness of switching between graphical and programmatic representation)

In [ ]:
from pyironflow.pyironflow import PyironFlow

wf = pf.get_workflow()

pf = PyironFlow([wf])
pf.gui

### Hash database

In [ ]:
from pyiron_workflow import Workflow   

import pyiron_nodes as pn

wf = Workflow('hash_db')
wf.db = pn.databases.node_hash_db.create_db()

pf = PyironFlow([wf]) #, hash_nodes=True)
pf.gui

In [ ]:
pf.out_widget

### Phonons

The following example computes the phonon total density of states. The dos node returns a dataframe. To plot it use from the Node Library the _PlotDataFrame_ module *(Node Library -> plotting -> PlotDataFrame)*

In [ ]:
from pyiron_workflow import Workflow   

import pyiron_nodes as pn

wf = Workflow('phonons')
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.CubicBulkCell('Pb', cell_size=3)
wf.phonopy = pn.atomistic.property.phonons.CreatePhonopy(structure=wf.bulk, engine=wf.engine) #, parameters=parameters)
wf.dos = pn.atomistic.property.phonons.GetTotalDos(phonopy=wf.phonopy.outputs.phonopy)

# wf.run()


In [ ]:
wf.dos.inputs.storage.value.hash_output

In [ ]:
pf = PyironFlow([wf], hash_nodes=True)
pf.gui

### Built Lammps workflow from scratch

This example demonstrates how to construct a low-level workflow to run a file-based executable using Lammps as a prototype. The concepts can be extended to any type of executable.

In [25]:
from pyiron_workflow import Workflow   
import pyiron_nodes as pn


In [26]:
# import pyiron_nodes.atomistic

wf = Workflow('Lammps')
wf.structure = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(structure=wf.structure, repeat_scalar=3)

wf.calculator = pn.atomistic.engine.lammps.CalcMD() # temperature=300, n_ionic_steps=10_000)

wf.run()

DEBUG:pyiron_log:Not supported parameter used!


,Parameter,Value,Comment
0,units,metal,
1,dimension,3,
2,boundary,p p p,
3,atom_style,atomic,
4,read_data,structure.inp,
5,include,potential.inp,
6,fix___ensemble,all nvt temp 300.0 300.0 0.1,
7,variable___dumptime,equal 100,
8,variable___thermotime,equal 100,
9,timestep,0.001,


In [27]:
wf = Workflow('Lammps')
wf.structure = pn.atomistic.structure.build.Bulk('Al', cubic=True)
wf.repeat = pn.atomistic.structure.transform.Repeat(structure=wf.structure, repeat_scalar=3)

wf.calculator = pn.atomistic.engine.lammps.CalcMD() # temperature=300, n_ionic_steps=10_000)
wf.potential = pn.atomistic.engine.lammps.Potential(
    structure=wf.structure, name='1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1'
)

wf.init_lammps = pn.atomistic.engine.lammps.InitLammps(
        structure=wf.repeat,
        potential=wf.potential,
        calculator=wf.calculator,
        working_directory="test2",
    )

wf.shell = pn.atomistic.engine.lammps.Shell(
        # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
        working_directory=wf.init_lammps,
    )

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(
    log_file=wf.shell.outputs.log
)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(
    dump_file=wf.shell.outputs.dump
)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode='md',
)

wf.get_energy_pot = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.Collect)

out = wf.run()
out

DEBUG:pyiron_log:Not supported parameter used!


array([-362.87999968, -358.5129855 , -358.4579939 , -358.31296039,
       -358.55784106, -359.02478623, -358.31887295, -358.83137511,
       -358.65856012, -358.41650426, -358.63303643, -358.63670514,
       -358.55424417, -358.60848674, -359.21976388, -358.75045594,
       -358.70085842, -358.09667742, -358.11768594, -358.43833566,
       -358.73646116, -359.09860407, -358.2410861 , -357.79137736,
       -357.97615099, -358.66342062, -358.23955412, -358.88205371,
       -358.83486582, -359.26242862, -358.55243496, -358.55474489,
       -358.21297665, -358.97124696, -358.55524676, -358.56762719,
       -359.29673694, -358.80134   , -358.24187177, -358.5103194 ,
       -358.91956568, -358.9306468 , -358.32669732, -358.33391415,
       -358.16457596, -358.60606464, -358.32974592, -359.15559159,
       -359.30010577, -359.47381373, -359.00845451, -358.39813831,
       -358.75495891, -357.87957696, -358.21394858, -358.55263975,
       -358.86352564, -358.93426393, -358.5099942 , -358.42666

In [28]:
wf.get_energy_pot.pull()

DEBUG:pyiron_log:Not supported parameter used!


array([-362.87999968, -358.5129855 , -358.4579939 , -358.31296039,
       -358.55784106, -359.02478623, -358.31887295, -358.83137511,
       -358.65856012, -358.41650426, -358.63303643, -358.63670514,
       -358.55424417, -358.60848674, -359.21976388, -358.75045594,
       -358.70085842, -358.09667742, -358.11768594, -358.43833566,
       -358.73646116, -359.09860407, -358.2410861 , -357.79137736,
       -357.97615099, -358.66342062, -358.23955412, -358.88205371,
       -358.83486582, -359.26242862, -358.55243496, -358.55474489,
       -358.21297665, -358.97124696, -358.55524676, -358.56762719,
       -359.29673694, -358.80134   , -358.24187177, -358.5103194 ,
       -358.91956568, -358.9306468 , -358.32669732, -358.33391415,
       -358.16457596, -358.60606464, -358.32974592, -359.15559159,
       -359.30010577, -359.47381373, -359.00845451, -358.39813831,
       -358.75495891, -357.87957696, -358.21394858, -358.55263975,
       -358.86352564, -358.93426393, -358.5099942 , -358.42666

In [29]:
from pyironflow.pyironflow import PyironFlow

pf = PyironFlow([wf])
pf.gui

In [ ]:
wf_new = pf.wf_widgets[0].get_workflow()

In [ ]:
wf_new = pf.get_workflow()

pf = PyironFlow([wf_new])
pf.gui

In [ ]:
wf_new.repeat.pull()

In [ ]:
wf.repeat.pull()

In [ ]:
wf._edges

In [ ]:
type(wf_new.ParseLogFile.inputs.log_file.value)

In [ ]:
type(wf.ParseLogFile.inputs.log_file.value)

### Use Lammps Macro

The low-level workflow above has been converted into a macro 'Code'. The application of such a high-level macro is shown below. 

In [ ]:
wf = Workflow('lammps_macro')
wf.bulk = pn.atomistic.structure.build.CubicBulkCell('Pb', cell_size=3)
wf.inp_calc_md = pn.atomistic.calculator.data.InputCalcMD()
wf.lammps = pn.atomistic.engine.lammps.Code(structure=wf.bulk, calculator=wf.inp_calc_md) 
wf.energies = pn.atomistic.engine.lammps.GetEnergyPot(generic=wf.lammps)
wf.plot = pn.plotting.Plot(y=wf.energies)

wf.run()

In [ ]:
pf = PyironFlow([wf])
pf.gui

In [ ]:
pf.wf_widgets[0].wf.bulk.outputs.structure.value.__reduce__();